In [165]:
import pandas as pd
import sklearn
from sklearn import preprocessing

In [169]:
train_data = pd.read_csv("data/train.csv",index_col=0)

#train_data = train_data.fillna(0)
train_data

,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,bid4,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
id,,,,,,,,,,,,,,,,,,,,,
0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,3841.0,...,1,6,14,6,6,1,1,10,2,1
1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,3842.0,...,6,11,1,6,1,4,4,1,13,0
2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,3843.0,...,1,4,21,12,1,16,10,4,9,0
3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,3842.0,...,13,12,2,4,2,7,1,2,11,1
4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,3841.8,...,12,2,2,4,1,3,1,11,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592375,4110.2,4110.3,1.0,1.0,2.0,1,4110.2,4110.0,4109.4,4109.2,...,2,1,3,2,2,3,10,7,7,1
592376,4109.4,4110.5,6.0,5.0,11.0,1,4109.2,4109.0,4108.6,4108.2,...,2,2,6,5,3,9,7,7,5,0
592377,4109.4,4110.5,NaN,NaN,0.0,0,4109.2,4109.0,4108.6,4108.2,...,2,2,6,5,3,9,7,7,5,0


In [163]:
for i in train_data["last_price"].values:
    if i==0:
        print(i)

In [137]:
features=train_data.iloc[:,:-1] 
labels = train_data.iloc[:,-1:]
features = features[["last_price", "d_open_interest", "bid1","ask1", "bid1vol", "ask1vol"]]
features["bid"]=features["bid1"]<=features["last_price"]
features["ask"]=features["ask1"]>=features["last_price"]
features = features[["d_open_interest", "bid1vol", "ask1vol", "bid", "ask"]]

In [150]:

'''
X = features.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
'''
X = preprocessing.scale(features.values)
X = features.values
Y = labels.values
Y = Y.reshape(len(Y))
x_train = X[:500000]
x_val = X[500000:]
y_train = Y[:500000]
y_val = Y[500000:]


In [132]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
lr = LogisticRegression(max_iter=100000).fit(x_train, y_train)
svc = LinearSVC(max_iter=100000)
ada = AdaBoostClassifier()
mlp = MLPClassifier(alpha=1, max_iter=1000)
lr.fit(x_train, y_train)
from sklearn.metrics import f1_score
f1_score(y_val, lr.predict(x_val))

0.15981027974058656

In [141]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(x_train, y_train)
model.score(x_val,y_val)

0.6289889586490582

In [142]:
f1_score(y_val, model.predict(x_val))

0.23048944768747195

In [156]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, model.predict_proba(x_val)[:,1])

0.6201340771761394

In [157]:
sum(model.predict(x_val))

9162